###3-1. 네이버 책 검색 API 호출하기 + 검색어로  찾은  책 목록을 json 파일로 저장하기

In [35]:
import requests
import pandas as pd
import json
import os
import re
from datetime import datetime

# 공통 헤더 설정 함수
def get_naver_api_headers():
    client_id = os.getenv('NAVER_CLIENT_ID')
    client_secret = os.getenv('NAVER_CLIENT_SECRET')
    
    if not client_id or not client_secret:
        raise ValueError("NAVER_CLIENT_ID 또는 NAVER_CLIENT_SECRET 환경 변수가 설정되지 않았습니다.")
    
    return {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }

# JSON 파일 저장 함수
def save_to_json(data, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# 책 검색 함수
def search_books(query):
    payload = {
        'query': query,
        'display': 30,
        'sort': 'sim'
    }
    
    url = 'https://openapi.naver.com/v1/search/book.json'
    headers = get_naver_api_headers()
    
    res = requests.get(url, params=payload, headers=headers)
    res.raise_for_status()
    
    return res.json()['items']

# 데이터 전처리 함수
def preprocess_book_data(books):
    for book in books:
        # 출판일 포맷 변경
        if 'pubdate' in book:
            try:
                pubdate = datetime.strptime(book['pubdate'], '%Y%m%d').strftime('%Y-%m-%d')
                book['pubdate'] = pubdate
            except:
                book['pubdate'] = '정보 없음'
        
        # 가격 포맷
        if 'discount' in book:
            try:
                discount = int(book['discount'])
                book['price'] = f"{discount:,}원"
            except:
                book['price'] = '정보 없음'
        
        # 설명 100자로 제한 + HTML 태그 제거
        if 'description' in book:
            clean = re.sub('<.*?>', '', book['description'])
            book['description'] = clean[:100] + ('...' if len(clean) > 100 else '')
    
    return books

# 결과 출력 함수
def print_books(books):
    print("=== 도서 목록 ===")
    for idx, book in enumerate(books, 1):
        print(f"{idx}. {book['title']} ({book['author']})")
        print(f"   저자: {book['author']}")
        print(f"   출판사: {book['publisher']}")
        print(f"   출판일: {book.get('pubdate', '정보 없음')}")
        print(f"   가격: {book.get('price', '정보 없음')}")
        print(f"   설명: {book.get('description', '설명 없음')}")
        print(f"   URL: {book.get('link', '링크 없음')}\n")


# 메인 실행 함수
def main():
    query = '파이썬'        #input("검색할 도서명을 입력하세요: ").strip()
    if not query:
        print("도서명을 입력해야 합니다.")
        return

    try:
        books = search_books(query)
        save_to_json(books, 'data/books.json')
        print("도서 정보가 data/books.json 파일에 저장되었습니다.")
        
        df = pd.DataFrame(books)
        print("\nDataFrame 정보:")
        print(df.info())
        
        processed_books = preprocess_book_data(books)
        print_books(processed_books)
        
    except Exception as e:
        print(f"오류 발생: {e}")

if __name__ == "__main__":
    main()


도서 정보가 data/books.json 파일에 저장되었습니다.

DataFrame 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        30 non-null     object
 1   link         30 non-null     object
 2   image        30 non-null     object
 3   author       30 non-null     object
 4   discount     30 non-null     object
 5   publisher    30 non-null     object
 6   pubdate      30 non-null     object
 7   isbn         30 non-null     object
 8   description  30 non-null     object
dtypes: object(9)
memory usage: 2.2+ KB
None
=== 도서 목록 ===
1. 혼자 만들면서 공부하는 파이썬 (37개 라이브러리와 API로 보고서, 차트, 게임, 맵, AI 챗봇 만들기|저자 직강 유튜브 강의, 오픈 채팅 제공) (문현일)
   저자: 문현일
   출판사: 한빛미디어
   출판일: 2025-02-14
   가격: 23,400원
   설명: “파이썬을 배워서 어디에 활용할 수 있을지 궁금했나요?”
나의 신용카드 내역 분석, 우리 아파트 실거래가 시각화해보기, 맛집 지도 웹 앱 만들기 등 
생활 속  15가지 파이썬 프...
   URL: https://search.shopping.naver.com/book/catalog/52859338841

2. 혼자 

### books.json 파일을 Pandas DataFrame로 저장하기


In [ ]:

# JSON 파일을 DataFrame으로 읽기
import pandas as pd

def load_books(filepath='data/books.json'):
    # JSON 파일 로드
    df = pd.read_json(filepath, encoding='utf-8')
    
    # 컬럼명 보기 좋게 변경
    df = df.rename(columns={
        'title': '제목',
        'author': '저자',
        'publisher': '출판사',
        'pubdate': '출판일',
        'discount': '가격(원)',
        'link': 'URL'
    })
    
    return df
load_books()

,상품명,상품URL,이미지URL,최저가,최고가,판매몰,상품ID,상품유형,브랜드,제조사,카테고리1,카테고리2,카테고리3,카테고리4
0,백화점3-5일배송 폴로 가디건 걸즈 걸스 랄프로렌 꽈배기 블랙 화이트 핑크 헌터네이비XL,https://smartstore.naver.com/main/products/667...,https://shopping-phinf.pstatic.net/main_842166...,64900,,나우인뉴욕,84216650390,2,폴로랄프로렌,폴로랄프로렌,패션의류,여성의류,카디건,
1,봄 가을 간절기 라운드 루즈핏 크롭 가디건 레드 그린 핑크 화이트 블랙 옐로우,https://smartstore.naver.com/main/products/546...,https://shopping-phinf.pstatic.net/main_830114...,19900,,ARUMY,83011442056,2,,,패션의류,여성의류,카디건,
2,부드러운 캐시미어 크롭 가디건 숏 라운드 니트가디건 여성 가디건 봄 간절기 레드 핑크,https://smartstore.naver.com/main/products/482...,https://shopping-phinf.pstatic.net/main_823726...,18900,,코이블리,82372678315,2,,,패션의류,여성의류,카디건,
3,[백화점3-5일배송] 폴로 가디건 니트 걸즈 랄프로렌 블랙 화이트 핑크 헌터네이비XL L,https://smartstore.naver.com/main/products/845...,https://shopping-phinf.pstatic.net/main_859949...,63900,,트위티 155,85994962324,2,폴로랄프로렌,폴로랄프로렌,패션의류,여성의류,카디건,
4,[국내발송 + 선물포장] 폴로 랄프로렌 여아 아기 코튼 베이비 가디건 핑크 옐로우 화이트,https://smartstore.naver.com/main/products/945...,https://shopping-phinf.pstatic.net/main_870040...,69000,,리틀데이즈,87004045841,2,폴로랄프로렌,폴로랄프로렌,출산/육아,유아동의류,카디건,
5,10COLOR 여성 봄 비비드컬러 라운드 슬릿넥 크롭 니트 그린 핑크가디건,https://smartstore.naver.com/main/products/995...,https://shopping-phinf.pstatic.net/main_875040...,24900,,ARUMY,87504043024,2,,,패션의류,여성의류,카디건,
6,[백화점] 폴로 가디건 걸즈 케이블 꽈배기 니트,https://smartstore.naver.com/main/products/111...,https://shopping-phinf.pstatic.net/main_886518...,69000,,세컨찬스라이프,88651894622,2,폴로랄프로렌,폴로랄프로렌,패션의류,여성의류,카디건,
7,꼼데가르송 기본와펜 가디건 여성용,https://smartstore.naver.com/main/products/474...,https://shopping-phinf.pstatic.net/main_822861...,254390,,레이바쿠,82286128596,2,꼼데가르송,,패션의류,여성의류,카디건,
8,제너럴아이디어 WOMAN GNRL 에센셜 베이직 카디건 11COL WBD3L04503,https://search.shopping.naver.com/catalog/5328...,https://shopping-phinf.pstatic.net/main_532872...,32400,,네이버,53287208769,1,제너럴아이디어,제너럴아이디어,패션의류,여성의류,카디건,
9,메종키츠네 가디건 베이비폭스 폭스헤드 더블폭스 칠랙스 [당일출고],https://smartstore.naver.com/main/products/530...,https://shopping-phinf.pstatic.net/main_828471...,267000,,브랜드리퍼블릭,82847151528,2,메종키츠네,메종키츠네,패션의류,여성의류,카디건,


### 한번더 검색어로  찾은  책 목록 출력하기

In [ ]:
import pandas as pd

# JSON 파일을 DataFrame으로 불러오기
book_df = load_books()

keyword = input("검색할 키워드를 입력하세요 : ").strip()

result = book_df.loc[book_df['제목'].str.contains(keyword)].reset_index(drop=True)
result if not result.empty else print(f"{keyword}에 해당하는 도서를 찾을 수 없습니다.")

### 검색어로  찾은  책 목록 중에서 가격이 2만원 이상인 책만 출력하기

In [ ]:

book_df.loc[book_df['가격(원)'] >= 20000,book_df.columns.drop(['URL','image','isbn','description'])].sort_values(by='가격(원)',ascending=True).reset_index(drop=True)

,제목,저자,가격(원),출판사,출판일
0,코딩 자율학습 나도코딩의 파이썬 입문 (초보자 눈높이에 맞춘 친절한 프로그래밍 자습서),나도코딩,21600,길벗,20230220
1,파이썬 (다양한 예제로 알로리즘 쉽게 분석하기),김유진,22500,다본,20190410
2,파이썬 프로그래밍 (기초부터 딥러닝까지),허준,22500,성안당,20250312
3,엑셀X파이썬 (엑셀이 파이썬을 품었다),정성일,22500,길벗,20240730
4,"혼자 만들면서 공부하는 파이썬 (37개 라이브러리와 API로 보고서, 차트, 게임,...",문현일,23400,한빛미디어,20250214
5,밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현),사이토 고키,23400,한빛미디어,20250124
6,파이썬,김영천,23750,기한재,20200225
7,파이썬 플러스,최희식,23750,인피니티북스,20240731
8,내공의 파이썬 (한 권으로 끝내는 초보자용 코딩 입문서),한선관,24300,자유아카데미,20250315
9,파이썬 + AI (코드조각과 실행화면으로 배우는 코딩),안용제^김남규,25200,디지털북스,20240715


### 검색어로  찾은  책 목록 중에서 출판사가 인피니티북스인 책만 출력하기

In [ ]:
book_df.loc[book_df['출판사'] == '인피니티북스',book_df.columns.drop(['image','description'])].reset_index(drop=True)

,제목,URL,저자,가격(원),출판사,출판일,isbn
0,파이썬,https://search.shopping.naver.com/book/catalog...,천인국,12000,인피니티북스,20170830,9791185578330
1,파이썬 플러스,https://search.shopping.naver.com/book/catalog...,최희식,23750,인피니티북스,20240731,9791192373362
